1. it is needed to import geographyical coordination
2. next read paths
3. calculate length of each path
4. read deamands
5. calclulate for each demand length 
6. lenght * 0.03 dB/km


In [278]:
import xml.etree.ElementTree as ET
import geopy.distance

my_tree = ET.parse("network.xml")
root = my_tree.getroot()
print(root[0][0][0][0][0].text)
print(root[0][0][0][0][1].text)

18.6
54.2


In [279]:
class Node:
    def __init__(self, name: str, x: float,  y: float):
        self.name = name 
        self.x = x
        self.y = y
    def get_coordinates(self):
        return (self.x, self.y)


In [280]:
nodes = []
for child in root[0][0]:
    nodes.append(Node(child.attrib["id"], float(child[0][0].text), float(child[0][1].text) ))
id = 0
for node in nodes:
    print(id, node.name, node.get_coordinates())
    id += 1

0 Gdansk (18.6, 54.2)
1 Bydgoszcz (17.9, 53.1)
2 Kolobrzeg (16.1, 54.2)
3 Katowice (18.8, 50.3)
4 Krakow (19.8, 50.0)
5 Bialystok (23.1, 53.1)
6 Lodz (19.4, 51.7)
7 Poznan (16.8, 52.4)
8 Rzeszow (21.9, 50.0)
9 Szczecin (14.5, 53.4)
10 Warsaw (21.0, 52.2)
11 Wroclaw (16.9, 51.1)


In [281]:
class Link:
    def __init__(self, link_id :str, start_node: Node, end_node: Node):
        self.link_id = link_id
        self.start_node = start_node
        self.end_node = end_node
        self.lenght = self.calculate_lenght()
    
    def calculate_lenght(self):
       return round(geopy.distance.geodesic(self.start_node.get_coordinates(), self.end_node.get_coordinates()).km , 0)

link_id is coded which city are connected eg 0 is Gdańsk, 10 is Warszawa Link_0_10

In [282]:
links = {}
for child in root[0][1]:
   start_node_id = int(child.attrib["id"].split("_")[1])
   end_node_id = int(child.attrib["id"].split("_")[2])
   links[child.attrib["id"]] = Link(child.attrib["id"], nodes[start_node_id], nodes[end_node_id])

print(links)


{'Link_0_10': <__main__.Link object at 0x000001FCDF48B3D0>, 'Link_0_2': <__main__.Link object at 0x000001FCDF48BEB0>, 'Link_1_2': <__main__.Link object at 0x000001FCDF48B5E0>, 'Link_1_7': <__main__.Link object at 0x000001FCDF48B4C0>, 'Link_1_10': <__main__.Link object at 0x000001FCDF48B9D0>, 'Link_2_9': <__main__.Link object at 0x000001FCDF45A0B0>, 'Link_3_4': <__main__.Link object at 0x000001FCDF1352A0>, 'Link_3_6': <__main__.Link object at 0x000001FCDF458A00>, 'Link_3_11': <__main__.Link object at 0x000001FCDF459480>, 'Link_4_8': <__main__.Link object at 0x000001FCDEEFABF0>, 'Link_4_10': <__main__.Link object at 0x000001FCDEEFB5E0>, 'Link_5_8': <__main__.Link object at 0x000001FCDEEFA980>, 'Link_5_10': <__main__.Link object at 0x000001FCDF2F9870>, 'Link_6_10': <__main__.Link object at 0x000001FCDF49C220>, 'Link_6_11': <__main__.Link object at 0x000001FCDF49C370>, 'Link_7_9': <__main__.Link object at 0x000001FCDF572620>, 'Link_7_11': <__main__.Link object at 0x000001FCDF571C60>, 'Link

In [283]:
class Path:
    def __init__(self, path_id, linksId):
        self.path_id = path_id
        self.linksId = linksId

class Demand:
    def __init__(self, name, start_node, end_node, paths):
        self.name = name
        self.start_node = start_node
        self.end_node = end_node
        self.paths = paths


The values power of laser are normalize based on article  Application of machine learningmethods  in provisioning  of DWDMchannels
Noise power is based on: https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&ved=2ahUKEwiNyIqopub3AhWw-yoKHQIrCKoQFnoECCMQAQ&url=https%3A%2F%2Fipnpr.jpl.nasa.gov%2Fprogress_report%2F42-58%2F58L.PDF&usg=AOvVaw2gKpN2HzVEWbv-nkDiOZSE
loss_per_km is well-know parameters
0 km = 50 OSNR


In [284]:
import math
loss_per_km = 0.2
P_signal = 40 # dBm
P_noise =  math.log( 4.2 * 10 ** -15, 10)# 4.2 * 10^-15
amplifier_gain = 40

def CalculateOSNR(distance):
    return P_signal - loss_per_km * distance - P_noise

# CalculateOSNR(1000)


In [285]:
demands = []
for child in root[1]:
    start_node_id = int(child.attrib["id"].split("_")[1])
    end_node_id = int(child.attrib["id"].split("_")[2])
    # child[3] admiissible_paths
    # print(child[3][1].attrib)
    paths = []
    for path in child[3]:
        # print(path.attrib)
        linksId = []
        for link in path:
            linksId.append(link.text)
            # print(linksId)
        paths.append( Path(path.attrib["id"], linksId))
    demands.append(Demand(child.attrib["id"], nodes[start_node_id], nodes[end_node_id], paths))


Jak Ci nie będzie pasował wektor danych to spóbuj go przerobić lub daj znać
demand can be deduced on source and dest
paths are aloweys 7
############################################################################
source_node, destination_node, distance, OSNR,

In [286]:
# data for 
data = [] # src, dst, distance, OSNR


for demand in demands:
    # print(demand.name.split('_'))
    src_node = demand.name.split('_')[1]
    dst_node = demand.name.split('_')[2]
    for path in demand.paths:
        # print(path.path_id, path.linksId)
        distance = 0
        number_of_amplifiers = 0
        for link in path.linksId:
            distance += links[link].calculate_lenght()
            number_of_amplifiers += 1
        new_row = [src_node, dst_node, distance, CalculateOSNR(distance) + number_of_amplifiers * amplifier_gain]
        data.append(new_row)
data

[['0', '1', 508.0, 32.77675070960209],
 ['0', '1', 694.0, -4.423249290397905],
 ['0', '1', 893.0, 35.776750709602084],
 ['0', '1', 1117.0, -49.0232492903979],
 ['0', '1', 1227.0, 48.9767507096021],
 ['0', '1', 1852.0, 3.976750709602072],
 ['0', '1', 1711.0, -87.82324929039794],
 ['0', '2', 277.0, 38.97675070960209],
 ['0', '2', 925.0, -10.623249290397894],
 ['0', '2', 1348.0, -55.223249290397916],
 ['0', '2', 1458.0, 42.776750709602084],
 ['0', '2', 1310.0, -7.623249290397894],
 ['0', '2', 1942.0, -94.02324929039793],
 ['0', '2', 1881.0, -1.8232492903979391],
 ['0', '3', 1205.0, -26.623249290397894],
 ['0', '3', 718.0, 30.776750709602112],
 ['0', '3', 1141.0, -13.82324929039791],
 ['0', '3', 1702.0, -46.02324929039793],
 ['0', '3', 1244.0, 5.576750709602095],
 ['0', '3', 685.0, 37.376750709602106],
 ['0', '3', 1108.0, -7.223249290397916],
 ['0', '4', 1090.0, -43.623249290397894],
 ['0', '4', 603.0, 13.776750709602084],
 ['0', '4', 1026.0, -30.82324929039791],
 ['0', '4', 1129.0, -11.42

In [287]:
test = []
for d in data:
    test.append(d[3])
print(sorted(test))

<class 'list'>
